# Module 3
Question 2

Should the Department of Housing Preservation and Development of New York City focus on any particular set of boroughs, ZIP codes, or street (where the complaints are severe) for the specific type of complaints you identified in response to Question 1?

### Import Libraries

In [1]:
from botocore.client import Config
import ibm_boto3
import pandas as pd

### Load Data
Load data collected in notebook_1 and convert to dataframe

In [3]:
client_cred.download_file(Bucket=bucket,Key='df_raw_cos.pkl',Filename='./df_raw_local.pkl')
df = pd.read_pickle('./df_raw_local.pkl')

### Inspect Data

In [4]:
df.head()

,created_date,unique_key,complaint_type,incident_zip,incident_address,street_name,address_type,city,resolution_description,borough,latitude,longitude,closed_date,location_type,status
0,2019-12-10T09:07:56.000,45131499,GENERAL,11226.0,90 EAST 18 STREET,EAST 18 STREET,ADDRESS,BROOKLYN,The following complaint conditions are still o...,BROOKLYN,40.648965,-73.962994,NaN,RESIDENTIAL BUILDING,Open
1,2019-12-10T03:53:20.000,45130500,PLUMBING,11691.0,20-33 SEAGIRT BOULEVARD,SEAGIRT BOULEVARD,ADDRESS,Far Rockaway,The following complaint conditions are still o...,QUEENS,40.595176,-73.755376,NaN,RESIDENTIAL BUILDING,Open
2,2019-12-10T16:17:03.000,45131649,HEAT/HOT WATER,10025.0,504 WEST 111 STREET,WEST 111 STREET,ADDRESS,NEW YORK,The following complaint conditions are still o...,MANHATTAN,40.803907,-73.964063,NaN,RESIDENTIAL BUILDING,Open
3,2019-12-10T22:09:16.000,45130383,HEAT/HOT WATER,11233.0,1572 EASTERN PARKWAY EXTENSION,EASTERN PARKWAY EXTENSION,ADDRESS,BROOKLYN,The following complaint conditions are still o...,BROOKLYN,40.671014,-73.916204,NaN,RESIDENTIAL BUILDING,Open
4,2019-12-10T18:30:00.000,45130428,HEAT/HOT WATER,11213.0,1338 BERGEN STREET,BERGEN STREET,ADDRESS,BROOKLYN,The following complaint conditions are still o...,BROOKLYN,40.675561,-73.940256,NaN,RESIDENTIAL BUILDING,Open


In [5]:
df['borough'].value_counts()

BROOKLYN         1720990
BRONX            1595479
MANHATTAN        1041447
Unspecified       873227
QUEENS            637198
STATEN ISLAND      86799
Name: borough, dtype: int64

In [6]:
df['complaint_type'].value_counts()[:11]

HEAT/HOT WATER          1215878
HEATING                  887869
PLUMBING                 708151
GENERAL CONSTRUCTION     500863
UNSANITARY CONDITION     446863
PAINT - PLASTER          361257
PAINT/PLASTER            344588
ELECTRIC                 305932
NONCONST                 260890
DOOR/WINDOW              203362
WATER LEAK               191595
Name: complaint_type, dtype: int64

## Clean Data

The below cells clean the dataset for analyzing incident by borough. Incidents where the borough is unspecified are removed, heating is combined with heat/hot water as one complaint. 

In [7]:
# get rid of the unspecified columns
df = df[df['borough'] != 'Unspecified']

In [8]:
# combine same complaints
df['complaint_type'] = df['complaint_type'].map(lambda x : "HEAT/HOT WATER" if x == 'HEATING' else x)

In [9]:
df = df[df['complaint_type'] == 'HEAT/HOT WATER']

## Analyze Data 

The below cells take a look at the distribution of heat / hot water complaints by various levels including zip, street, & borough. Goal is to identify if location is something that should be a looked into as a potential cause for complaint.

In [10]:
print("The Number of heat related complaints by Borough:")
df['borough'].value_counts(normalize= True)

The Number of heat related complaints by Borough:


BRONX            0.323942
BROOKLYN         0.308667
MANHATTAN        0.226993
QUEENS           0.130535
STATEN ISLAND    0.009863
Name: borough, dtype: float64

In [11]:
print("The Number of heat related complaints by City:")
df['city'].value_counts(normalize = True)[:11]

The Number of heat related complaints by City:


BRONX            0.323972
BROOKLYN         0.308785
NEW YORK         0.226776
Jamaica          0.011897
Elmhurst         0.010720
Flushing         0.010554
Astoria          0.010389
STATEN ISLAND    0.009875
Far Rockaway     0.006409
Ridgewood        0.006080
Woodside         0.005887
Name: city, dtype: float64

In [12]:
print("The Number of heat related complaints by Zip:")
df['incident_zip'].value_counts(normalize=True).head(10)

The Number of heat related complaints by Zip:


11226.0    0.032352
10467.0    0.030969
10458.0    0.030873
10468.0    0.027329
10453.0    0.027044
10452.0    0.023472
10457.0    0.021473
10031.0    0.021315
10040.0    0.020733
10456.0    0.020589
Name: incident_zip, dtype: float64

In [14]:
print("The Number of heat related complaints by Street Name:")
df['street_name'].value_counts()[:11]

The Number of heat related complaints by Street Name:


GRAND CONCOURSE             32477
BROADWAY                    21318
OCEAN AVENUE                15531
MORRIS AVENUE               13907
ARDEN STREET                13762
ST NICHOLAS AVENUE          13027
ELMHURST AVENUE             11633
BOYNTON AVENUE              10529
AMSTERDAM AVENUE            10493
BAILEY AVENUE                9252
DR M L KING JR BOULEVARD     8901
Name: street_name, dtype: int64

The below cells look at the distribution of severe complaints for the Bronx

In [15]:
df_bronx = df[df['borough'] == 'BRONX']

In [16]:
df_bronx['street_name'].value_counts(normalize=True)[:11]

GRAND CONCOURSE             0.055060
MORRIS AVENUE               0.023577
BOYNTON AVENUE              0.017851
BAILEY AVENUE               0.015686
DR M L KING JR BOULEVARD    0.015090
DECATUR AVENUE              0.014097
WALTON AVENUE               0.013863
SEDGWICK AVENUE             0.013821
CRESTON AVENUE              0.013068
WALLACE AVENUE              0.011289
SHERIDAN AVENUE             0.011062
Name: street_name, dtype: float64

In [17]:
df_bronx['incident_zip'].value_counts(normalize=True).head(10)

10467.0    0.095586
10458.0    0.095290
10468.0    0.084350
10453.0    0.083472
10452.0    0.072447
10457.0    0.066276
10456.0    0.063548
10462.0    0.058885
10463.0    0.050654
10472.0    0.050070
Name: incident_zip, dtype: float64

The below cells look at the distribution of severe complaints for Brooklyn

In [18]:
df_bk = df[df['borough'] == 'BROOKLYN']

In [19]:
df_bk['street_name'].value_counts(normalize=True)[:11]

OCEAN AVENUE        0.027632
OCEAN PARKWAY       0.015613
LINDEN BOULEVARD    0.013757
NOSTRAND AVENUE     0.013693
EASTERN PARKWAY     0.012159
BEDFORD AVENUE      0.012110
ROCKAWAY PARKWAY    0.010099
FULTON STREET       0.009927
NEW YORK AVENUE     0.009544
ST JOHNS PLACE      0.009409
KINGS HIGHWAY       0.009243
Name: street_name, dtype: float64

In [20]:
df_bk['incident_zip'].value_counts(normalize=True).head(10)

11226.0    0.104766
11225.0    0.065289
11213.0    0.061481
11212.0    0.052245
11233.0    0.045964
11203.0    0.044189
11221.0    0.041323
11216.0    0.039429
11207.0    0.038800
11208.0    0.035368
Name: incident_zip, dtype: float64

## Export Data

In [21]:
# convert file to .pkl 
df_bk.to_pickle('./df_bk_raw.pkl')

# upload to cloud for use in later notebooks
client_cred.upload_file('./df_bk_raw.pkl',bucket,'df_bk_raw_cos.pkl')

## Conclusion

Brooklyn has more gross complaints where the Bronx has the most complaints we identified as severe in nature, but not by a wide margin. The zip code 11226 has 10% of the total complaints in Brooklyn occurring in that area. Will look further into this relationship for Brooklyn in the next notebook.